In [ ]:
import pandas as pd
import json

In [ ]:
data = []
with open(r'indosum\dev.01.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

with open(r'indosum\test.01.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

with open(r'indosum\train.01.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)
df.head()

In [ ]:
ringkas = df[['category', 'source', 'source_url', 'paragraphs', 'summary']]
ringkas.head()

In [ ]:
data = []
for ringkasan in df['summary']:
    teks = ' '.join([' '.join(kalimat).replace(' ,', ',').replace(' .', '.') for kalimat in ringkasan])
    data.append(teks)

ringkas['ringkasan']= data
ringkas.head()

In [ ]:
del ringkas['summary']
ringkas.head()

In [ ]:
#simpan ringkas ke excel
# ringkas.to_excel('indosum.xlsx', index=False)

In [ ]:
url = ringkas['source_url'][0]
print(url)

# Menggunakan split() untuk membagi URL berdasarkan tanda "/"
bagian = url.split("/")

# Memilih bagian terakhir setelah tanda "/" terakhir
kalimat_terakhir = bagian[-1].replace("-", " ")

print(kalimat_terakhir)


In [ ]:
for jdl in ringkas['source_url']:
    print(jdl)
    if jdl.split("/")[-1] == '':
        print(jdl.split("/")[-2].replace("-", " ").replace(".html", ""))
    else:
        print(jdl.split("/")[-1].replace("-", " ").replace(".html", ""))


In [ ]:
import requests
from bs4 import BeautifulSoup

for url in ringkas['source_url']:
    print(url)
    try:
        # Mengirim permintaan GET ke URL
        response = requests.get(url)

        # Mengecek apakah permintaan berhasil
        response.raise_for_status()

        # Membuat objek BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

       # Mengambil elemen judul halaman jika ada
        judul_elem = soup.title
        if judul_elem:
            judul = judul_elem.text.strip()
            print(judul)
        else:
            print("Error: Tidak dapat menemukan judul halaman")

    except requests.exceptions.RequestException as e:
        print("Error: Tidak dapat mengakses URL")


In [ ]:
#baca indosum.xlsx
import pandas as pd
ringkas = pd.read_excel('indosum.xlsx')
ringkas.head()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Specify the GPU device index

import requests
from bs4 import BeautifulSoup
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

JudulONL = []
for url in ringkas['source_url']:
    try:
        # Mengirim permintaan GET ke URL
        response = requests.get(url)

        # Mengecek apakah permintaan berhasil
        response.raise_for_status()

        # Membuat objek BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Mengambil elemen judul halaman jika ada
        judul_elem = soup.title
        if judul_elem:
            ttl = judul_elem.text.strip()
            JudulONL.append(ttl)
            print(ttl)
        else:
            if url.split("/")[-1] == '':
                JudulONL.append(url.split("/")[-2].replace("-", " ").replace(".html", ""))
            else:
                JudulONL.append(url.split("/")[-1].replace("-", " ").replace(".html", ""))

    except requests.exceptions.RequestException as e:
        if url.split("/")[-1] == '':
            JudulONL.append(url.split("/")[-2].replace("-", " ").replace(".html", ""))
        else:
            JudulONL.append(url.split("/")[-1].replace("-", " ").replace(".html", ""))

JudulONL = [torch.tensor(judul, device=device) for judul in JudulONL]  # Move data to GPU

ringkas['Judul Online'] = JudulONL
ringkas.head()


In [ ]:
!nvidia-smi